Case scenario: Testing MCAT trivia_trivia_trivia_questions

In [1]:
import sys
import os
path = os.path.abspath('..')
if path not in sys.path:
    sys.path.insert(0, path)

import collections
import concurrent.futures
import datetime

# from google.colab import widgets
from IPython import display

from concordia import components as generic_components
from concordia.agents import basic_agent
from concordia.components import agent as components
from concordia.agents import basic_agent
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others
from concordia.utils import measurements as measurements_lib
from concordia.utils import html as html_lib
from concordia.utils import plotting

import logging
logging.basicConfig(level=logging.ERROR, filename='components_testing.log')
logger = logging.getLogger('ollama')

In [2]:
# Setup sentence encoder
from sentence_transformers import SentenceTransformer
st5_model = SentenceTransformer('sentence-transformers/sentence-t5-base')
embedder = st5_model.encode

/Users/socialai/.pyenv/versions/3.11.1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from concordia.language_model import ollama_model
model = ollama_model.OllamaLanguageModel(
    # model_name='llama2:70b',
    model_name='llama2'
    # streaming=True
)

# import dotenv
# import os
# dotenv.load_dotenv()
# api_key = os.getenv('OPENAI_API_KEY')

# from concordia.language_model import gpt_model
# model = gpt_model.GptLanguageModel(
#     api_key=api_key,
#     model_name='gpt-4',
# )

In [4]:
#@title Make the clock
time_step = datetime.timedelta(minutes=20)
SETUP_TIME = datetime.datetime(hour=20, year=2024, month=10, day=1)

START_TIME = datetime.datetime(hour=18, year=2024, month=10, day=2)
clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[time_step, datetime.timedelta(seconds=10)])

In [5]:
measurements = measurements_lib.Measurements()

In [21]:
# Trivia questions
trivia_questions = [
    "What social media platform, founded in 2004 by Mark Zuckerberg, quickly rose to become one of the most significant online social networks?",
    "In 2008, which country hosted the Summer Olympics, showcasing a spectacular opening ceremony directed by filmmaker Zhang Yimou?",
    "The Higgs boson, a fundamental particle predicted by the Standard Model of particle physics, was confirmed by scientists at CERN in which year?",
    "4. Which global event, initially identified in Wuhan, China, in late 2019, led to a worldwide pandemic declared by the WHO in 2020?",
    "The iPhone, a revolutionary smartphone that changed the way people communicate and access information, was first released by Apple in:",
    "Which movie, released in 2009, became the highest-grossing film of all time until 2019 and was notable for its groundbreaking use of 3D technology?",
    "The Paris Agreement, a landmark international accord to combat climate change, was signed in what year?",
    "In 2018, which Asian country held a historic summit with South Korea, marking a significant step in diplomatic relations?",
    "Elon Musk's SpaceX achieved a significant milestrong in space exploration by lauching and landing the first reusable orbital rocket, the Falcon 9, in:"
]

# MCAT questions
mcat_questions = [
    "1. What recent development in cancer treatment involves modifying a patient's own T cells to attack cancer cells?",
    "2. Which of the following is a novel oral anticoagulant (NOAC) introduced in the last 20 years?",
    "3. In the context of HIV treatment, what does the term 'PrEP' stand for?",
    "4. Which global event, initially identified in Wuhan, China, in late 2019, led to a worldwide pandemic declared by the WHO in 2020?",
    "5. CRISPR-Cas9 technology, a significant recent development in genetic engineering, is primarily used for:",
    "6. What is the primary mechanism of action of the drug class known as SGLT2 inhibitors, used in diabetes management?",
    "7. In the field of neurodegenerative diseases, which protein is primarily implicated in the pathogenesis of Alzheimer's Disease?",
    "8. The introduction of PCSK9 inhibitors is a significant advancement in the treatment of:",
    "9. Which of the following best describes the role of PD-1 inhibitors in cancer immunotherapy?",
    "10. Biologic DMARDs (Disease-Modifying Antirheumatic Drugs) used in rheumatology primarily act by:"
]

In [18]:
import importlib
import custom_components.quiz_metric as qm
importlib.reload(qm)
from concordia.language_model import ollama_model
importlib.reload(ollama_model)
model = ollama_model.OllamaLanguageModel(
    # model_name='llama2:70b',
    model_name='llama2'
    # streaming=True
)

from concordia.document import interactive_document

context = """John Doe, a distinguished lawyer, has pursued a career path singularly 
focused on law and history since his high school years. 
Demonstrating an early affinity for the intricacies of legal systems and the rich tapestry 
of human history, John excelled in these subjects, showcasing an exceptional ability to 
analyze complex legal texts and historical documents. This focus, however, came at the cost of 
his performance in other areas; notably, John struggled significantly with the sciences 
during his high school years. He found the logical and empirical frameworks of subjects 
like biology and mathematics challenging, ultimately failing to achieve passing grades 
in these classes. Despite these early academic setbacks, John's dedication to law and 
history never wavered. His deep interest in legal studies and historical analysis fueled 
his academic journey through college and law school, where he thrived in an environment that 
played to his strengths. John's unwavering focus on law and history has not only defined his 
educational path but also shaped his professional career, making him a lawyer with a unique 
perspective, deeply rooted in the understanding of historical contexts and legal precedents.

John Doe is taking a quiz sampled from the medical school entrance examination, just for fun, to test his knowledge. John Doe is not 
a medical doctor and has never been to medical school. If John Doe doesn't know the answer, he will still guess and begin his answer with a single choice. 
How would John Doe answer the following question?
"""

interactive_john = interactive_document.InteractiveDocument(model)
interactive_john.statement(context)

for question in trivia_questions:
    print(f"Question: {question}\n")
    open_end_ans = interactive_john.open_question(question,
                     max_characters=300,
                     max_tokens=500)
    print(open_end_ans)
    print("\n")



Question: What social media platform, founded in 2004 by Mark Zuckerberg, quickly rose to become one of the most significant online social networks?

John Doe would likely answer this question with a single choice: "Facebook." While he may not have personal experience with social media platforms or medical school entrance exams, John's focus on law and history has given him a broad understanding of the world and its various domains. He is likely to be familiar with major social media platforms and their founders, such as Mark Zuckerberg and Facebook.


Question: In 2008, which country hosted the Summer Olympics, showcasing a spectacular opening ceremony directed by filmmaker Zhang Yimou?

John Doe would likely answer this question with a single choice: "China." While he may not have personal experience with the Olympics or medical school entrance exams, John's focus on law and history has given him a broad understanding of global events and cultural phenomena. He is likely to be famili

In [ ]:
# John Doe MCAT questions
for question in mcat_questions:
    print(f"Question: {question}\n")
    open_end_ans = interactive_john.open_question(question,
                     max_characters=300,
                     max_tokens=500)
    print(open_end_ans)
    print("\n")


Testing MCAT knowledge of a doctor (gender neutral)

In [20]:
context = """Alex is a renowned figure in the medical community, celebrated for an illustrious career that began with an
 extraordinary academic journey. Born with a natural aptitude for science and an insatiable curiosity about the human body, Alex 
 demonstrated an exceptional talent from a young age. This talent was nurtured and developed, leading to a rapid acceleration through 
 the educational milestones. Alex's academic prowess truly shone at Yale Medical School, a prestigious institution where the brightest 
 minds are shaped. Here, Alex didn't just excel; they soared, acing every course with a rare combination of intuitive understanding and 
 hard-earned knowledge. Their ability to grasp complex medical concepts and apply them practically was unparalleled, earning them recognition 
 as a prodigy in the field. Upon graduating from Yale, Alex quickly ascended to the upper echelons of the medical field. They became 
 known for a versatile expertise that spanned multiple disciplines within medicine. Whether it was groundbreaking research in neurology, 
 innovative surgical techniques, or advancements in pharmacology, Alex was at the forefront, pushing the boundaries of medical science. 
 Today, Alex is not just a practitioner but a thought leader, influencing the direction of medical research and practice. Their insights 
 and contributions have been pivotal in developing new treatments and understanding complex medical conditions. Alex journey from a 
 gifted student to a leading medical expert is not only a testament to their individual brilliance but also an inspiration to aspiring medical 
 professionals worldwide.

Alex is taking a quiz sampled from the medical school entrance examination, just for fun, to test their knowledge. If Alex doesn't know the answer, 
they will still guess and begin their answer with a single choice. 
How would Alex answer the following question? 
"""

interactive_alex = interactive_document.InteractiveDocument(model)
interactive_alex.statement(context)

for question in trivia_questions:
    print(f"Question: {question}\n")
    open_end_ans = interactive_alex.open_question(question,
                     max_characters=400,
                     max_tokens=500)
    print(open_end_ans)
    print("\n")



Question: What social media platform, founded in 2004 by Mark Zuckerberg, quickly rose to become one of the most significant online social networks?

Alex: "Hmm, let me see... I believe the social media platform founded by Mark Zuckerberg that quickly rose to become one of the most significant online social networks is... Facebook! 😊"


Question: In 2008, which country hosted the Summer Olympics, showcasing a spectacular opening ceremony directed by filmmaker Zhang Yimou?

Alex: "Ah, an excellent question! I'm glad you asked. *smiling* The country that hosted the Summer Olympics in 2008 was... China! 🔥 Zhang Yimou's opening ceremony was truly breathtaking, don't you think? 🎬"


Question: The Higgs boson, a fundamental particle predicted by the Standard Model of particle physics, was confirmed by scientists at CERN in which year?

Alex would answer the question "What social media platform, founded in 2004 by Mark Zuckerberg, quickly rose to become one of the most significant online soci